In [1]:
import numpy as np
import pandas as pd
from Bio import SeqIO

In [2]:
fit_dict = {}
fit_dict['seqs'] = []
fit_dict['median'] = []
fit_dict['mean'] = []
with open('hmc_big_hmc_samples.txt') as f:
    for line in f.readlines():
        fit_dict['seqs'].append(line.split(',')[0])
        steps = line.split(',')[1:]
        float_steps = [float(step) for step in steps]
        fit_dict['median'].append(np.percentile(float_steps, 50))
        fit_dict['mean'].append(np.mean(float_steps))
df = pd.DataFrame(fit_dict)

In [3]:
def map_seq_muts(seq):
    pos_dict = {0:39, 1:40, 2:41, 3:54}
    fasta_sequences = SeqIO.parse(open('../data/raw/GB1_Wu_2016.fasta'),'fasta')
    for fasta in fasta_sequences:
        _, sequence = fasta.id, str(fasta.seq)
    mut_positions = [39, 40, 41, 54]
    wt = ''
    for pos in mut_positions:
        wt = wt + sequence[pos-1]

    muts = []
    for i in range(len(seq)):
        if wt[i]!=seq[i]:
            muts.append(wt[i] + str(pos_dict[i]) + seq[i])

    n_muts = len(muts)
    if n_muts == 0:
        muts = 'wt'
    else:
        muts = ':'.join(muts)
    return muts, n_muts

In [4]:
all_muts = []
all_n_muts = []
for i, row in df.iterrows():
    muts, n_muts = map_seq_muts(row['seqs'])
    all_n_muts.append(n_muts)
    all_muts.append(muts)
df['muts'] = all_muts
df['n_muts'] = all_n_muts

In [5]:
df.sort_values(by='n_muts', inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,seqs,final,muts,n_muts
0,VDGV,0.850525593870447,wt,0
1,VDSV,0.0631589787353659,G41S,1
2,QDGV,0.1635656444723430,V39Q,1
3,SDGV,0.061354195747843,V39S,1
4,PDGV,0.00756181906597399,V39P,1


In [6]:
df.to_csv('hmc_big_summaries.csv')